In [ ]:
## Setting up CNN for classifying badminton shots

In [1]:
# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import SGD,RMSprop,adam
from keras import callbacks

PATH = os.getcwd()
data_path = './data/'
data_dir_list = os.listdir(data_path)
print(data_dir_list)

Using TensorFlow backend.


['backhand', 'defense', 'drop', 'smash']


In [2]:
# Processing input dataset
img_rows = 128
img_cols = 128
num_channel = 1
num_epoch = 20

# Define the number of classes
num_classes = 4
labels_name = {'smash':0,'drop':1,'defense':2,'backhand':3}

img_data_list = []
labels_list = []

for dataset in data_dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    print("Class: " + dataset)
    print("Images found: " + str(len(img_list)))
    print ('Loading the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    counter = 0
    for img in img_list:
        input_img = cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize = cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)
        labels_list.append(label)
        counter += 1
        if counter % 10 == 0:
            cv2.destroyAllWindows()
            print("Frames processed: " + str(counter) + " in " + dataset)
#             cv2.imshow("Frame: " + img, input_img_resize)
#             cv2.waitKey(500)
cv2.destroyAllWindows()

Class: backhand
Images found: 133
Loading the images of dataset-backhand

Frames processed: 10 in backhand
Frames processed: 20 in backhand
Frames processed: 30 in backhand
Frames processed: 40 in backhand
Frames processed: 50 in backhand
Frames processed: 60 in backhand
Frames processed: 70 in backhand
Frames processed: 80 in backhand
Frames processed: 90 in backhand
Frames processed: 100 in backhand
Frames processed: 110 in backhand
Frames processed: 120 in backhand
Frames processed: 130 in backhand
Class: defense
Images found: 186
Loading the images of dataset-defense

Frames processed: 10 in defense
Frames processed: 20 in defense
Frames processed: 30 in defense
Frames processed: 40 in defense
Frames processed: 50 in defense
Frames processed: 60 in defense
Frames processed: 70 in defense
Frames processed: 80 in defense
Frames processed: 90 in defense
Frames processed: 100 in defense
Frames processed: 110 in defense
Frames processed: 120 in defense
Frames processed: 130 in defense
F

In [6]:
print(len(img_data_list))
print(len(labels_list))

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

labels = np.array(labels_list)

# print the count of number of samples for different classes
print(np.unique(labels,return_counts=True))

# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

1162
1162
(1162, 128, 128)
(array([0, 1, 2, 3]), array([360, 483, 186, 133], dtype=int64))


In [7]:
img_data = np.expand_dims(img_data, axis=4) 
print (img_data.shape)

# Shuffle the dataset
x, y = shuffle(img_data,Y, random_state=2)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

(1162, 128, 128, 1)


C:\Users\swlee\Anaconda3\envs\cmpe257_project\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """Entry point for launching an IPython kernel.


In [8]:
# Defining the model
input_shape = img_data[0].shape


model = Sequential()

model.add(Conv2D(32,(3,3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3), padding='same', activation='relu'))
model.add(Conv2D(256,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.0001), metrics=['accuracy'])

'''
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape = input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
#model.add(Convolution2D(64, 3, 3))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])
'''

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model.summary()
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape
model.layers[0].output_shape
model.layers[0].get_weights()
np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable

In [9]:
filename='model_train_new.csv'
csv_log = callbacks.CSVLogger(filename, separator=',', append=False)

early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')

filepath = "Best-weights-cnn-1-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"

checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [csv_log, early_stopping, checkpoint]

hist = model.fit(X_train, y_train, batch_size=4, nb_epoch=num_epoch, verbose=1, validation_data=(X_test, y_test),callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.


C:\Users\swlee\Anaconda3\envs\cmpe257_project\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 929 samples, validate on 233 samples
Epoch 1/20
929/929 [==============================] - 9s 10ms/step - loss: 1.3135 - acc: 0.4650 - val_loss: 1.0939 - val_acc: 0.6309

Epoch 00001: val_loss improved from inf to 1.09393, saving model to Best-weights-cnn-1-001-1.3135-0.4650.hdf5
Epoch 2/20
929/929 [==============================] - 5s 6ms/step - loss: 0.8005 - acc: 0.7169 - val_loss: 0.4233 - val_acc: 0.8755

Epoch 00002: val_loss improved from 1.09393 to 0.42330, saving model to Best-weights-cnn-1-002-0.8005-0.7169.hdf5
Epoch 3/20
929/929 [==============================] - 5s 6ms/step - loss: 0.5280 - acc: 0.8299 - val_loss: 0.3321 - val_acc: 0.9185

Epoch 00003: val_loss improved from 0.42330 to 0.33207, saving model to Best-weights-cnn-1-003-0.5280-0.8299.hdf5
Epoch 4/20
929/929 [==============================] - 5s 6ms/step - loss: 0.5130 - acc: 0.8439 - val_loss: 0.5187 - val_acc: 0.8927

Epoch 00004: val_loss did not improve from 0.33207
Epoch 5/20
929/929 [============

In [10]:
score = model.evaluate(x=X_test, y=y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

233/233 [==============================] - 0s 2ms/step
Test Loss: 0.2767593262661168
Test accuracy: 0.9098712446351931


In [11]:
!mkdir -p saved_model
model.save('saved_model/shot_classifier_2')

A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


In [12]:
from keras.models import load_model

#new_model = Model()
new_model = load_model('saved_model/shot_classifier_2')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 64)        18496     
__________

In [22]:
X_test = X_test[:1]
new_model.predict_classes(X_test, batch_size=32, verbose=1)

1/1 [==============================] - 0s 4ms/step


array([2], dtype=int64)

In [13]:
X_test.shape

(233, 128, 128, 1)

In [20]:
X_test[:1].shape

(1, 128, 128, 1)